# TripPal Search Engine 
this search engine provides integration between different data APIs; like `Dojo Hotel` and `OpenTripMap` 

In [7]:
import pandas as pd
import numpy as np
import requests 
import json
import math
from utils import hotels_engine as hotels
from utils import places_engine as places 
from utils import data_parser as dp
from utils import constants as c

### Section One - Hotels search engine 

In [8]:
def search_location(loc, pn, check_in, check_out, adults, sort_order = 'PRICE', filters = {}):
    '''
    sort_order : One of the following is allowed: 
    BEST_SELLER|
    STAR_RATING_HIGHEST_FIRST|
    STAR_RATING_LOWEST_FIRST|
    DISTANCE_FROM_LANDMARK|
    GUEST_RATING|
    PRICE_HIGHEST_FIRST|
    PRICE
    '''    
    idx = hotels.get_city_id(loc)
    
    res = hotels.get_hotels_by_des_id(idx, pn, check_in, check_out, adults, sort_order , filters)
    
    df = hotels.get_search_results(res)
    
#     df = dp.clean_results(df)
    
    jsn = hotels.get_search_filters(res)
    
#     return {'HOTELS' : dp.get_json_file(df),
#             'FILTERS' : jsn}
    return df

In [10]:
res = search_location('istanbul', 1, "2021-03-16", "2021-03-20", 1)

In [11]:
res.head(2)

,id,name,starRating,urls,address,guestReviews,landmarks,geoBullets,ratePlan,neighbourhood,deals,messaging,badging,pimmsAttributes,coordinate,providerType,supplierHotelId,vrBadge,isAlternative,optimizedThumbUrls
0,645103552,Gökçe Pansiyon - Adult Only,0.0,{},"{'streetAddress': 'Civelek Sokak', 'extendedAd...","{'unformattedRating': 6.4, 'rating': '6.4', 't...","[{'label': 'City center', 'distance': '4.9 mil...",[],"{'price': {'current': '$4', 'exactCurrent': 3....",Kagithane,{},{},{},DoubleStamps|D13|TESCO,"{'lat': 41.07552, 'lon': 28.98155}",LOCAL,20128236,Hostel,False,{'srpDesktop': 'https://thumbnails.trvl-media....
1,1223523520,Nostalji Hostel,0.0,{},{'streetAddress': 'Taksim Dolapdere Cadde 107'...,"{'unformattedRating': 7.0, 'rating': '7.0', 't...","[{'label': 'City center', 'distance': '2.8 mil...",[],"{'price': {'current': '$5', 'exactCurrent': 5....",Harbiye,{},{},{},DoubleStamps|D13,"{'lat': 41.045963, 'lon': 28.983801}",LOCAL,38203860,NaN,False,{'srpDesktop': 'https://thumbnails.trvl-media....


In [7]:
def best_hotels_in_country(country):
    return  search_location(country, 
                            1, 
                            "2020-01-01",
                            "2021-01-01", 1,
                            'STAR_RATING_HIGHEST_FIRST' ,
                            {"guestRatingMin":"8"})

In [8]:
co = best_hotels_in_country('china')

In [11]:
co.head(1)

,id,name,starRating,urls,address,guestReviews,landmarks,geoBullets,ratePlan,neighbourhood,deals,messaging,badging,pimmsAttributes,coordinate,providerType,supplierHotelId,isAlternative,optimizedThumbUrls,vrBadge
0,186370,Wanda Vista Beijing,5.0,{},"{'streetAddress': 'Tower C Wanda Plaza, 93 Jia...","{'unformattedRating': 8.6, 'rating': '8.6', 't...","[{'label': 'City center', 'distance': '3.8 mil...",[],"{'price': {'current': '$123', 'exactCurrent': ...",Jianwai Street,{},{},{},DoubleStamps|D13|TESCO,"{'lat': 39.907605, 'lon': 116.463455}",MULTISOURCE,1697569,False,{'srpDesktop': 'https://thumbnails.trvl-media....,NaN


In [4]:
def hotel_details(destination_id : int):
    
    response = hotels.get_hotel_details(destination_id)

    return dp.parse_details(response)

In [5]:
# dp.get_json_file(res)
det = hotel_details(453028)

In [6]:
det

{'HOTEL_FEATURE': ['32 smoke-free guestrooms',
  'Daily housekeeping',
  'Business center',
  'Terrace',
  '24-hour front desk',
  'Coffee/tea in a common area',
  'Air conditioning',
  'Garden',
  'Computer station',
  'Meeting room',
  'Multilingual staff',
  'Picnic area',
  'Free WiFi and free parking '],
 'LOCATION_SECTION': ['In East Williamsburg',
  'Brooklyn Bridge Park - 3.2 mi / 5.1 km',
  'Prospect Park - 3.6 mi / 5.7 km',
  'Barclays Center Brooklyn - 3.9 mi / 6.3 km',
  'Washington Square Park - 4.5 mi / 7.3 km',
  'New York University - 4.6 mi / 7.3 km',
  'Union Square Park - 4.8 mi / 7.8 km',
  'Wall Street - 4.9 mi / 8 km',
  'Chelsea Market - 5.4 mi / 8.7 km',
  'One World Trade Center - 5.5 mi / 8.8 km',
  'The High Line Park - 5.9 mi / 9.5 km'],
 'TAGLINE': ['<b>No-frills hostel in East Williamsburg </b>'],
 'HOTEL_FREEBIES': ['Free WiFi and free parking'],
 'NAME': 'NY Moore Hostel',
 'STAR_RATE': 2.0,
 'PRICE': 68.0,
 'longitude': -73.937509,
 'latitude': 40.70417

In [33]:
df = pd.DataFrame(data = det['HOTEL_FEATURE'])
df.head(10)

,0
0,32 smoke-free guestrooms
1,Daily housekeeping
2,Business center
3,Terrace
4,24-hour front desk
5,Coffee/tea in a common area
6,Air conditioning
7,Garden
8,Computer station
9,Meeting room


In [53]:
dicter = {}

In [27]:
det['HOTEL_FEATURE']

['32 smoke-free guestrooms',
 'Daily housekeeping',
 'Business center',
 'Terrace',
 '24-hour front desk',
 'Coffee/tea in a common area',
 'Air conditioning',
 'Garden',
 'Computer station',
 'Meeting room',
 'Multilingual staff',
 'Picnic area',
 'Free WiFi and free parking ']

In [54]:
res_str = ''
for row in det['HOTEL_FEATURE']:
    res_str += row
    res_str += ', '

dicter.update({'HOTEL_FEATURE' : res_str})

In [55]:
res_str = ' '
for idx in det['ROOM_FEATURES']:
    for item in idx['listItems']:
        res_str += item
        res_str += ', '
        
dicter.update({'ROOM_FEATURES' : res_str})

In [56]:
res_str = ' '
for idx in det['HOTEL_FEATURES']:
    for item in idx['listItems']:
        res_str += item
        res_str += ', '
dicter.update({'HOTEL_FEATURES' : res_str})

In [78]:
dicter.update({'NAME' : det['NAME']})

In [79]:
dfr = pd.DataFrame(data = dicter ,index = [0],
                 )

In [80]:
dfr.head()

,HOTEL_FEATURE,ROOM_FEATURES,HOTEL_FEATURES,NAME
0,"32 smoke-free guestrooms, Daily housekeeping, ...","Air conditioning, Soundproofed rooms, Individ...","Coffee/tea in a common area, Business center,...",NY Moore Hostel


In [8]:
rev = hotels.get_hotel_reviews(453028, 1)

In [9]:
imgs = hotels.get_hotel_imgs(453028)

In [10]:
idx = hotels.get_hotel_id_by_name('Albrechtshof')

In [11]:
idx

'799081408'

### Section two - Places search engine 

In [14]:
places.get_list_of_places('new yourk', 'churches')

[{'xid': 'W93406623',
  'name': "Saint Peter's Episcopal Church",
  'rate': '3h',
  'img_url': 'https://commons.wikimedia.org/wiki/File:St_Peters_Church_2011.jpg'},
 {'xid': 'W93282521',
  'name': "St Mary's Roman Catholic Church",
  'rate': '3h',
  'img_url': 'https://commons.wikimedia.org/wiki/File:StMarysChurchAlbany.jpg'},
 {'xid': 'W194233571',
  'name': 'New Jerusalem Home of the Saved Church',
  'rate': '1'},
 {'xid': 'W93282551', 'name': 'First Dutch Reformed Church', 'rate': '1'},
 {'xid': 'Q5452657',
  'name': 'First Church in Albany',
  'rate': '3h',
  'img_url': 'https://commons.wikimedia.org/wiki/File:Dutch_Church_Albany.jpg'},
 {'xid': 'W105511227',
  'name': "Saint Joseph's Park",
  'rate': '3',
  'img_url': 'https://commons.wikimedia.org/wiki/File:Saint_Josephs_Church_Albany.jpg'},
 {'xid': 'W126310802',
  'name': "Saint Joseph's Church",
  'rate': '3',
  'img_url': 'https://commons.wikimedia.org/wiki/File:Saint_Josephs_Church_Albany.jpg'},
 {'xid': 'W90702210',
  'name

In [12]:
places.get_object_properties('W331188177')

{'xid': 'W331188177',
 'name': 'Bıçakçı Alaaddin Cami',
 'address': {'city': 'İskenderpaşa Mahallesi',
  'road': 'Molla Hüsrev Caddesi',
  'town': 'Fatih',
  'state': 'Marmara Bölgesi',
  'country': 'Türkiye',
  'postcode': '34096',
  'country_code': 'tr'},
 'rate': '1',
 'osm': 'way/331188177',
 'bbox': {'lon_min': 28.949586,
  'lon_max': 28.94984,
  'lat_min': 41.0137,
  'lat_max': 41.01391},
 'kinds': 'religion,other_temples,interesting_places',
 'sources': {'geometry': 'osm', 'attributes': ['osm']},
 'otm': 'https://opentripmap.com/en/card/W331188177',
 'point': {'lon': 28.949715, 'lat': 41.013805}}

In [8]:
pls = places.search_places('istanbul')
pls['features'][1]

{'type': 'Feature',
 'id': '4850575',
 'geometry': {'type': 'Point', 'coordinates': [28.94873, 41.013432]},
 'properties': {'xid': 'W523880524',
  'name': 'Sofular Molla Hüsrev Camii',
  'dist': 90.52688443,
  'rate': 1,
  'osm': 'way/523880524',
  'kinds': 'religion,other_temples,interesting_places'}}

### Collecting Places Dataset
**get_place_features from places.py** rreturns the following features for each place in a wanted location:
 1.  `ID`
 2.  `NAME`
 3.  `RATE`
 4.  `Kinds` 
 
a search query to find all places in **Istanbul** for example, the first result is:
`{'id': 'W331188177',
 'name': 'Bıçakçı Alaaddin Cami',
 'rate': 1,
 'kinds': 'religion,other_temples,interesting_places'}`

In [51]:
test = places.get_place_features('istanbul','religion')
df = pd.DataFrame(test)

In [52]:
df.to_csv('dataset/religion_dataset.csv',index = False)

In [53]:
df2 = pd.read_csv('dataset/final_places_dataset.csv')

In [54]:
cities_name = df2.city.unique()

In [56]:
for city in cities_name:
    print(city)
    test = places.get_place_features(city,'religion' )
    df_ = pd.DataFrame(test)
    df_.to_csv('dataset/religion_dataset.csv', mode='a', header=False, index = False)

Milan
Madrid
Barcelona
Valencia
Rome
Naples
Venice
Palermo
Berlin
Hamburg
Frankfurt
Vienna
London
Singapore
Paris
New York
Kuala Lumpur
Delhi
Shenzhen
Mumbai
Phuket
Tokyo
Pattaya
Taipei
Dubai
Mecca
abu dhabi
Sharjah
Ajman
Fujairah
Dammam
Medina
Riyadh
Beirut
Byblos
Zahle
Damascus
Aleppo
Homs
Latakia
Tartus‎
Hama‎
Cairo‎
Giza‎
Luxor‎
Aswan‎
Sousse‎
Kairouan‎
Tunis‎
Marrakesh‎
Fes‎
Manama
İstanbul
Ankara
Antalya
İzmir
Şanlıurfa
Bodrum
Marseille
Bordeaux
Nice
Amsterdam
Lisbon
Athens
Bangkok
Bali
OSAKA
SEOUL
Hong Kong
Las Vegas
Shanghai
Los Angeles
Orlando
Moscow
Dublin
Toronto
Sydney
Munich
Jakarta
Beijing
Penang Island
San Francisco
Honolulu
Hurghada
Batam
Mexico
São Paulo
Montreal
Washington
Chicago
Boston
San Jose
Tehran
Houston
Manila
Geneva
Colombo
Xiamen
Atlanta
Casablanca
Amman
Dallas
Lagos
Durban
Baghdad
Mosul
istanbul


In [57]:
df = pd.read_csv('dataset/religion_dataset.csv')

In [58]:
df = df.drop_duplicates(subset=['name'])

In [59]:
df = df.dropna(subset=['name'])

In [60]:
df.head()

,city,id,name,rate,kinds,distance,coordinates
0,istanbul,W331188177,Bıçakçı Alaaddin Cami,1,"religion,other_temples,interesting_places",6.144909,"{41.013805, 28.949715}"
1,istanbul,W523880524,Sofular Molla Hüsrev Camii,1,"religion,other_temples,interesting_places",90.526884,"{41.013432, 28.94873}"
2,istanbul,W201141056,Kızılminare Cami,1,"religion,other_temples,interesting_places",175.069318,"{41.01244, 28.950613}"
3,istanbul,W523880163,Yayla Kambur Mustafa Efendi Cami,1,"religion,other_temples,interesting_places",178.737372,"{41.015354, 28.948938}"
4,istanbul,W523879940,Ahmediye Cami,2,"religion,mosques,interesting_places",278.478759,"{41.014961, 28.946699}"


In [61]:
len(df.index)

29459

In [62]:
df.to_csv('dataset/religion_dataset.csv',index = False)

In [63]:
df1 = pd.read_csv('dataset/cultural_dataset.csv')
df2 = pd.read_csv('dataset/religion_dataset.csv')
df3 = pd.read_csv('dataset/historic_dataset.csv')
df4 = pd.read_csv('dataset/architecture_dataset.csv')
df5 = pd.read_csv('dataset/natural_dataset.csv')

In [64]:
df1 = df1.append(df2, ignore_index=True)
df1 = df1.append(df3, ignore_index=True)
df1 = df1.append(df4, ignore_index=True)
df1 = df1.append(df5, ignore_index=True)

In [65]:
df1 = df1.drop_duplicates(subset=['name'])
df1 = df1.drop_duplicates(subset=['id'])
df1 = df1.dropna(subset=['name'])

In [66]:
len(df1.index)

107675

In [67]:
df1.to_csv('dataset/final_places_dataset2.csv',index = False)